In [11]:
#import required libraries
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azureml.core import Workspace
import mlflow

#Enter details of your Azure Machine Learning workspace
subscription_id = '80c77c76-74ba-4c8c-8229-4c3b2957990c'
resource_group = 'sonata-test-rg'
workspace = 'sonata-test-ws'
#ws = Workspace.from_config()

#connect to the workspace
#ml_client = MLClient(DefaultAzureCredential(), Workspace.from_config())
ml_client = MLClient(DefaultAzureCredential(), subscription_id, resource_group, workspace)

ws = Workspace(subscription_id, resource_group, workspace)

mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

In [26]:
print(ws.get_mlflow_tracking_uri())

azureml://southcentralus.api.azureml.ms/mlflow/v2.0/subscriptions/80c77c76-74ba-4c8c-8229-4c3b2957990c/resourceGroups/sonata-test-rg/providers/Microsoft.MachineLearningServices/workspaces/sonata-test-ws?


In [12]:
from azure.ai.ml.entities import AmlCompute

# specify aml compute name.
cpu_compute_target = "cpu-cluster"

try:
    ml_client.compute.get(cpu_compute_target)
except Exception:
    print("Creating a new cpu compute target...")
    compute = AmlCompute(
        name=cpu_compute_target, size="STANDARD_D2_V2", min_instances=0, max_instances=4
    )
    ml_client.compute.begin_create_or_update(compute).result()

In [13]:
from azure.ai.ml import command, Input

# define the command
command_job = command(
    code="./",
    command="python Bert.py",
    #--cnn_dailymail ${{inputs.cnn_dailymail}}",
    environment="gpt2-venv:6", #"EnvTest:1",
    # inputs={
    #     "cnn_dailymail": Input(
    #         type="uri_file",
    #         path="https://datasets-server.huggingface.co/rows?dataset=cnn_dailymail&config=3.0.0&split=validation&offset=0&limit=5",
    #     )
    # },
    compute="cpu-cluster",
)

In [14]:
# submit the command
returned_job = ml_client.jobs.create_or_update(command_job)
# get a URL for the status of the job
returned_job.studio_url

Your file exceeds 100 MB. If you experience low speeds, latency, or broken connections, we recommend using the AzCopyv10 tool for this file transfer.

Example: azcopy copy '/mnt/batch/tasks/shared/LS_root/mounts/clusters/pooja-gpu-stanadardnc6/code/Users/v-poojr/MLFlow_B/Bert' 'https://sonatatestws5184162462.blob.core.windows.net/azureml-blobstore-e7095216-e923-42b8-bfc1-2d65afe0b4b7/LocalUpload/c2333167e1b742f1d6fbeff6ad26117f/Bert' 

See https://docs.microsoft.com/azure/storage/common/storage-use-azcopy-v10 for more information.
Uploading Bert (878.12 MBs): 100%|██████████| 878120828/878120828 [00:05<00:00, 166271084.12it/s]




'https://ml.azure.com/runs/orange_room_807thh32kr?wsid=/subscriptions/80c77c76-74ba-4c8c-8229-4c3b2957990c/resourcegroups/sonata-test-rg/workspaces/sonata-test-ws&tid=72f988bf-86f1-41af-91ab-2d7cd011db47'